In [1]:
import rasterio
from rasterio.transform import from_bounds
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import requests
from matplotlib.colors import ListedColormap
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import os

# ------------------------------------------------------------
data_in_geoserver_folder = "../geoserver_data/data"
data_folder = "../fuzzy-script/data"
os.makedirs(data_folder, exist_ok=True)
os.makedirs(data_in_geoserver_folder, exist_ok=True)

# ------------------------------------------------------------
# Define area and layers
# ------------------------------------------------------------
bbox = (8.3, 47.2, 8.75, 47.55)
width, height = 512, 512  # resolution

range_per_layer = {
    "pm10": [0,26],
    "pm25": [0,14],
    "no2": [0,60],
    "ozon": [0,170],
    "russ": [0,2],
    "laerm": [0,70]
}

layers = {
    "pm10": "https://wms.zh.ch/AwelLHPM10JahreZHWMS",
    "pm25": "https://wms.zh.ch/AwelLHPM25JahreZHWMS",
    "no2": "https://wms.zh.ch/AwelLHNO2JahreZHWMS",
    "ozon": "https://wms.zh.ch/AwelLHMP98JahreZHWMS",
    "russ": "https://wms.zh.ch/ImmissionenZHWMS",
    "laerm": "https://wms.geo.admin.ch/"
}

year = 2023
russ_year = 2020
layer_names = {
    "pm10": f"pm10-jahre-{year}",
    "pm25": f"pm25-jahre-{year}",
    "no2": f"no2-jahre-{year}",
    "ozon": f"mp98-jahre-{year}",
    "russ": f"bc-{russ_year}",
    "laerm": "ch.bafu.laerm-strassenlaerm_tag"
}

# ------------------------------------------------------------
# Download function (direct WMS for zh layers, else raster)
# ------------------------------------------------------------
def fetch_wms_image(url, layer_name, bbox, width, height, crs="EPSG:4326"):
    if "wms.zh.ch" in url:
        # Use direct image from WMS
        params = {
            "LAYERS": layer_name,
            "SERVICE": "WMS",
            "VERSION": "1.3.0",
            "REQUEST": "GetMap",
            "FORMAT": "image/png; mode=8bit",
            "CRS": "EPSG:2056",
            "BBOX": "2680000,1243000,2696931,1255698",  # fixed full Zurich bbox for PM10
            "WIDTH": 1600,
            "HEIGHT": 1200
        }
        r = requests.get(url, params=params)
        r.raise_for_status()
        img = Image.open(BytesIO(r.content)).convert("L")
        local_path = os.path.join(data_folder, f"{layer_name}.png")
        img.save(local_path)
        arr = np.array(img).astype(float)
        arr = np.nan_to_num(arr, nan=0)
        # Resize to match workflow resolution
        arr = np.array(Image.fromarray(arr).resize((width, height)))
        return arr
    else:
        # Other WMS or sources (default)
        params = {
            "service": "WMS",
            "version": "1.3.0",
            "request": "GetMap",
            "layers": layer_name,
            "bbox": f"{bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]}" if "geo.admin.ch" in url else f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
            "width": width,
            "height": height,
            "crs": crs,
            "format": "image/png",
            "transparent": "true"
        }
        r = requests.get(url, params=params)
        r.raise_for_status()
        img = Image.open(BytesIO(r.content)).convert("L")
        local_path = os.path.join(data_folder, f"{layer_name}.png")
        img.save(local_path)
        arr = np.array(img).astype(float)
        arr = np.nan_to_num(arr, nan=0)
        return arr

# ------------------------------------------------------------
# Download all layers
# ------------------------------------------------------------
print("Downloading layers...")
data = {}
for key in layers:
    print(f"  -> {key}")
    arr = fetch_wms_image(layers[key], layer_names[key], bbox, width, height)
    print(f"for {key} max is {np.max(arr)} and min is {np.min(arr)}.")
    layer_min, layer_max = range_per_layer[key]
    img_min, img_max = np.min(arr), np.max(arr)
    # Map grayscale -> concentration
    arr_scaled = layer_min + (arr - img_min) / (img_max - img_min + 1e-6) * (layer_max - layer_min)
    arr_norm = (arr_scaled - range_per_layer[key][0]) / (range_per_layer[key][1] - range_per_layer[key][0])
    data[key] = arr_norm

# ------------------------------------------------------------
# Fuzzy logic setup
# ------------------------------------------------------------
pm25 = ctrl.Antecedent(np.linspace(0, 1, 100), 'pm25')
pm10 = ctrl.Antecedent(np.linspace(0, 1, 100), 'pm10')
no2 = ctrl.Antecedent(np.linspace(0, 1, 100), 'no2')
ozon = ctrl.Antecedent(np.linspace(0, 1, 100), 'ozon')
russ = ctrl.Antecedent(np.linspace(0, 1, 100), 'russ')
laerm = ctrl.Antecedent(np.linspace(0, 1, 100), 'laerm')
quality = ctrl.Consequent(np.linspace(0, 100, 100), 'quality')


# good, moderate or bad
# PM25 ->   0 – 5   | 5 – 15    | > 15
# PM10 ->   0 – 10  | 10 – 20   | > 20
# NO2  ->   0 - 10  | 10 - 30   | > 30
# Ozon ->   0 – 100 | 100 – 160 | > 160
# Russ ->   0 – 1   | 1 – 1.7   | > 1.7
# Laerm ->  < 50    | 50 – 60   | > 60

max = 1
quality_per_layer = {
    "pm10": [0.7,0.8],
    "pm25": [0.5,1],
    "no2": [10/1000,30/1000],
    "ozon": [100/1000,160/1000],
    "russ": [1/1000,1.7/1000],
    "laerm": [50/1000,60/1000],
}
for var in [pm25, pm10, no2, ozon, russ, laerm]:
    low, high = quality_per_layer[var.label]
    var['good'] = fuzz.trimf(var.universe, [0, 0, low])
    var['moderate'] = fuzz.trimf(var.universe, [low, (low + high)/2, high])
    var['bad'] = fuzz.trimf(var.universe, [high, (high + max) / 2, max])
    quality['very_good'] = fuzz.trimf(quality.universe, [0, 0, 20])
quality['good'] = fuzz.trimf(quality.universe, [15, 30, 45])
quality['liveable'] = fuzz.trimf(quality.universe, [40, 55, 70])
quality['bad'] = fuzz.trimf(quality.universe, [65, 80, 90])
quality['very_bad'] = fuzz.trimf(quality.universe, [85, 100, 100])

rules = [
    ctrl.Rule(pm10['bad'], quality['very_bad']),
    ctrl.Rule(pm10['moderate'], quality['liveable']),
    ctrl.Rule(pm10['good'], quality['very_good']),
]

quality_ctrl = ctrl.ControlSystem(rules)
quality_sim = ctrl.ControlSystemSimulation(quality_ctrl)

# ------------------------------------------------------------
# Apply fuzzy logic
# ------------------------------------------------------------
h, w = data['pm25'].shape
fuzzy_output = np.zeros((h, w))

for i in range(h):
    for j in range(w):
        #quality_sim.input['pm25'] = data['pm25'][i, j]
        quality_sim.input['pm10'] = data['pm10'][i, j]
        #quality_sim.input['no2'] = data['no2'][i, j]
        #quality_sim.input['ozon'] = data['ozon'][i, j]
        #quality_sim.input['russ'] = data['russ'][i, j]
        #quality_sim.input['laerm'] = data['laerm'][i, j]
        try:
            quality_sim.compute()
            fuzzy_output[i, j] = quality_sim.output['quality']
        except:
            fuzzy_output[i, j] = np.nan

# ------------------------------------------------------------
# Export GeoTIFF
# ------------------------------------------------------------
output_path = os.path.join(data_in_geoserver_folder, "zurich_fuzzy_quality.tif")
transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], w, h)

with rasterio.open(
    output_path,
    "w",
    driver="GTiff",
    height=h,
    width=w,
    count=1,
    dtype=fuzzy_output.dtype,
    crs="EPSG:4326",
    transform=transform,
) as dst:
    dst.write(fuzzy_output, 1)

print(f"Saved fuzzy environmental quality map as: {output_path}")

# ------------------------------------------------------------
# 7️⃣ Visualize without tile borders
# ------------------------------------------------------------
norm_output = (fuzzy_output - np.nanmin(fuzzy_output)) / (np.nanmax(fuzzy_output) - np.nanmin(fuzzy_output))
classes = np.zeros_like(norm_output)
classes[(norm_output < 0.2)] = 1
classes[(norm_output >= 0.2) & (norm_output < 0.4)] = 2
classes[(norm_output >= 0.4) & (norm_output < 0.6)] = 3
classes[(norm_output >= 0.6) & (norm_output < 0.8)] = 4
classes[(norm_output >= 0.8)] = 5

cmap = ListedColormap(["darkgreen", "lightgreen", "yellow", "orange", "red"])
plt.figure(figsize=(10, 10))
plt.imshow(classes, cmap=cmap, extent=[bbox[0], bbox[2], bbox[1], bbox[3]])
plt.title("Fuzzy Environmental Quality Map of Zürich")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.colorbar(ticks=[1,2,3,4,5], label="1=Very Good → 5=Very Bad")
plt.show()


  -> pm10
for pm10 max is 231.6799774169922 and min is 110.43179321289062.
  -> pm25
for pm25 max is 232.39466857910156 and min is 117.09136199951172.
  -> no2
for no2 max is 227.34603881835938 and min is 67.8942642211914.
  -> ozon
for ozon max is 94.55255889892578 and min is 74.44557189941406.
  -> russ
for russ max is 225.80780029296875 and min is 75.89862060546875.
  -> laerm
for laerm max is 253.0 and min is 0.0.
